In [2]:
from collections import Counter
import pandas as pd
import numpy as np
import tqdm
import pickle
import matplotlib.pyplot as plt
from Bio.PDB import *
from rdkit import Chem
import os
import time
import shutil
from Bio.SeqUtils import seq1
import gzip
import tarfile
import sys
import random
from rdkit.Chem import rdFreeSASA
from rdkit import Chem
from rdkit.Chem.Descriptors import MolWt
import pybel
from rdkit.Chem.EnumerateStereoisomers import EnumerateStereoisomers

In [9]:
### 0. Prepare structure 373 for docking
### Important: structure has already been prepared in pocketvec_Murd.ipynb

### GET SMILES FROM LIGANDS
path = "/aloy/home/acomajuncosa/MurD/GitHub/osa-murd/data/structures"
os.chdir(path)
frags = ['349', "373", "374", "378"]

for frag in frags:
    if frag == '349':
        command = 'obabel ' + frag + "_ligand_only.pdb" + " -osmi > frags.smi"
    else:
        command = 'obabel ' + frag + "_ligand_only.pdb" + " -osmi >> frags.smi"
    os.system(command)

In [54]:
# Are all molecules flat?  YES THEY ARE

# molecules = pybel.readfile('sdf', '/aloy/home/acomajuncosa/MurD/GitHub/osa-murd/data/docking/full_lib/CDI_BMS_DivSet_300k_CleanSlate.sdf')
# coordinates = []

# for mol in tqdm.tqdm(molecules):
#     atoms = np.array([i.coords for i in mol.atoms])[:,2]  # Take all the z coords
#     coordinates.extend(atoms)
    
# print(Counter(coordinates))

In [55]:
### 1. SDF to smiles using obabel (it takes ~3min)
path_to_dataset = "../data/docking/full_lib/CDI_BMS_DivSet_300k_CleanSlate.sdf.gz"
path_to_output = "../data/docking/full_lib/smiles.smi"

command = "obabel " + path_to_dataset + " -osmi > " + path_to_output
os.system(command)

0

In [2]:
# ### 1. Take smiles directly from SDF (it takes ~2min)
# sdf = pybel.readfile('sdf', '../data/docking/full_lib/CDI_BMS_DivSet_300k_CleanSlate.sdf.gz')
# smiles = [[i.data['BMS (canonical SMILES)'], i.data['IDNUMBER']] for i in tqdm.tqdm(sdf)]

# with open("../data/docking/full_lib/smiles.smi", "w") as f:
#     for s in smiles:
#         f.write("\t".join(s) + "\n")

In [3]:
smiles = pd.read_csv("../data/docking/full_lib/smiles.smi", sep='\t', names=['smiles', 'id'])
smiles

,smiles,id
0,CC1(C)Cc2c(C#N)c(nc(c2CO1)N1CCOCC1)SCC(=O)Nc1c...,D396-0181
1,CC(CBr)n1ncnn1,8009-0167
2,CCCCS(=O)(=O)N=C1NCN(CC(=O)[O-])CN1,5044-0010
3,C[C@@]12CC[C@H](O)C[C@H]1[C@@H](O)CC1[C@H]2C[C...,N037-0036
4,Cc1[nH]c(=S)[nH]c(=O)c1CCCO,3399-0213
...,...,...
300534,CCCCNC(=O)C1(CCC1)c1ccc(cc1)NS(=O)(=O)c1cc(OC)...,L426-0625
300535,CCCCN(CC)c1ccc(cc1C(=O)O)NS(=O)(=O)c1cc(F)c(F)cc1,F294-0550
300536,CN(C)CCN(CC)c1ccc(cc1NS(=O)(=O)c1ccc(C)cc1C)C(...,F295-0500
300537,Nc1c(Br)cc(cc1Br)NS(=O)(=O)c1ccccc1,R052-1647


In [4]:
mols = [Chem.MolFromSmiles(i) for i in tqdm.tqdm(smiles['smiles'], position=0)]
smiles['nonisomeric'] = [Chem.MolToSmiles(i, isomericSmiles=False) for i in tqdm.tqdm(mols)]
del mols

100%|██████████| 300539/300539 [00:29<00:00, 10125.12it/s]


In [1]:
# # Inchies if needed
# mols = [Chem.MolFromSmiles(i) for i in tqdm.tqdm(smiles['nonisomeric'], position=0)]
# inchi = [Chem.MolToInchi(i) for i in mols]

# smiles['inchi'] = inchi

In [5]:
with open("/aloy/home/acomajuncosa/MurD/GitHub/osa-murd/data/docking/full_lib/nonisomeric_smiles.smi", "w") as f:
    for i, j in zip(smiles['nonisomeric'], smiles['id']):
        f.write("\t".join([i, j]) + "\n")

In [44]:
noniso_smiles = np.array([[i, j] for i, j in zip(smiles['nonisomeric'], smiles['id'])])
noniso_smiles = np.array_split(noniso_smiles, 950)

In [4]:
path = '/aloy/home/acomajuncosa/MurD/GitHub/osa-murd/data/docking/batches_smi'

for c, sm in enumerate(noniso_smiles):
    
    smile = sm[:,0]
    ids = sm[:,1]
    
    with open(os.path.join(path, "smiles" + "0"*(3-len(str(c))) + str(c)), "w") as f:
        
        for i, j in zip(smile, ids):
            f.write("\t".join([i, j]) + "\n")

NameError: name 'noniso_smiles' is not defined

In [6]:
in_path = '/aloy/home/acomajuncosa/MurD/GitHub/osa-murd/data/docking/batches_smi'
out_path = "/aloy/home/acomajuncosa/MurD/GitHub/docking/kraken/ligand_preparation"

for file in sorted(os.listdir(in_path)):
    
    numb = file
    
    if os.path.exists(os.path.join(out_path, numb)) is False: os.makedirs(os.path.join(out_path, numb))
    if os.path.exists(os.path.join(out_path, "out_" + numb)) is False: os.makedirs(os.path.join(out_path, "out_" + numb))
    shutil.copyfile(os.path.join(in_path, file), os.path.join(out_path, numb, file))   